In [1]:
%matplotlib notebook

In [2]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from numpy import std, pad, amax, abs
from numpy.random import normal
from numpy.fft import fftshift, ifftshift, fft2, ifft2 
from cv2 import imshow as cv2_imshow
#from google.colab.patches import cv2_imshow

In [3]:
# Filenames
filename = './pics/lenna.jpg'
test_filenames = ['./pics/barraxx.bmp', './pics/BDBOB.jpg',
                  './pics/pru2old.bmp']

# Show original image

In [34]:
original = cv.imread(filename=filename, flags=cv.IMREAD_COLOR)
cv.imshow(winname = 'original', mat = original)
cv.waitKey(0)
cv.destroyAllWindows()

# Blur + Noise

In [35]:
def noisy_gauss(image, snr=20):
      row, col, ch = image.shape
      sigma_image = std(image)
      sigma_noise = np.sqrt(sigma_image**2 * 10**(-snr/10))
      noise = normal(loc=0.0, scale=sigma_noise, size=image.shape)
      noisy = image + noise
      noisy = np.where(noisy < 0,  0, noisy)
      noisy = np.where(noisy > 255, 255, noisy)
      noisy = noisy.astype(dtype=np.uint8)
      return noisy, sigma_noise

In [36]:
kernel_shape = (5, 5)
lenna_blur = cv.blur(src=original, ksize=(5, 5))
lenna_blur_noise, noise_sigma = noisy_gauss(image=lenna_blur, snr=10)

In [37]:
cv2_imshow('lena blured and noisy',lenna_blur_noise)
cv.waitKey(0)
cv.destroyAllWindows()

# Wiener

In [40]:
kernel = 1 / kernel_shape[0] / kernel_shape[1]
kernel *= np.ones(shape=kernel_shape)

In [41]:
cv.split(lenna_blur_noise)

[array([[145, 138, 151, ..., 108, 113, 117],
        [150, 117, 150, ..., 121, 120, 113],
        [127, 143, 132, ...,  84,  82,  88],
        ...,
        [ 30,  35,  12, ...,  55,  27,  68],
        [  3,   0,   8, ...,  71,  89,  63],
        [ 28,  36,  26, ...,  82,  52,  29]], dtype=uint8),
 array([[115, 138, 128, ..., 122, 107, 117],
        [ 95, 129, 154, ..., 127,  97, 112],
        [142, 137, 124, ...,  72, 102,  53],
        ...,
        [ 10,  40,  42, ...,  37,  39,  48],
        [ 35,  12,   8, ...,  58,  57,  59],
        [ 21,  42,  41, ...,  56,  45,  53]], dtype=uint8),
 array([[163, 167, 109, ..., 125, 128, 141],
        [137, 122, 133, ..., 125, 138,  96],
        [132, 134, 126, ...,  80, 100,  78],
        ...,
        [ 54,  29,  31, ...,  14,  28,  54],
        [ 40,  55,  45, ...,  61,  45,  41],
        [ 34,  43,  35, ...,  33,  77,  55]], dtype=uint8)]

In [42]:
def recover(img, h, sn=None, wiener_inverse=False, show = True):
    
  #preparing H  
  a, b, _ = img.shape
  padded_kernel = np.zeros(shape=(a, b))
  c, d = h.shape
  padded_kernel[a//2 - c//2:a//2 + c//2 +1, b//2 - d//2: b//2 + d//2 +1] = h
  centered_fft2_kernel = fftshift(fft2(padded_kernel))
  #centered_fft2_kernel += 1.2
  H = centered_fft2_kernel

  channels = []
  for channel in cv.split(img):  #channel represents each R, B and G channel.
    channel_fft = fftshift(fft2(channel))
    if wiener_inverse:
      sf = channel_fft
      wiener_filter = np.divide(np.multiply(H.conjugate(), sf), np.multiply((np.abs(H))**2, sf) + sn ) )
      clean_lena = np.multiply(chanel_fft, wiener_filter)
      #clean_lena = np.multiply(np.divide(channel_fft, H), np.divide(np.abs(H)**2, (np.abs(H)**2 + np.divide(sn,sf))))
    else:
      clean_lena = np.divide(channel_fft, H)

    # filtered image ifft2 and normalize to unity
    clean_lena = np.abs(ifft2(ifftshift(clean_lena)))
    maximum = amax(clean_lena)
    clean_lena /= maximum
    clean_lena *= 255
    clean_lena = clean_lena.astype(np.uint8)
    channels.append(clean_lena)

  if len(channels) > 1:
    clean_lena = cv.merge(channels)
  else:
    clean_lena = channels
  newww = np.hstack((clean_lena, img))
  if show:
        cv2_imshow('recovered <LEFT>, original <RIGHT>', newww)
        cv.waitKey(0)
        cv.destroyAllWindows()
  return clean_lena

SyntaxError: invalid syntax (<ipython-input-42-b08309efc1ad>, line 17)

In [10]:
noise_shape = (lenna_blur_noise.shape[0], lenna_blur_noise.shape[1])
noise = np.ones(shape=noise_shape) * noise_sigma**2

wiener_result = recover(img=lenna_blur_noise, h=kernel, sn=noise, wiener_inverse=True)

# Inverse Filtering

In [11]:
inverse_result = recover(img=lenna_blur_noise, h=kernel, wiener_inverse=False)

In [12]:
stacked = np.hstack((wiener_result, inverse_result))
cv2_imshow('wiener vs inverse_simple', stacked)
cv.waitKey(0)
cv.destroyAllWindows()
print('Wiener vs Inverse')

Wiener vs Inverse


In [13]:
def estimate_blur_kernel(img):
  #estimated = None
  estimated = (1/9)*np.ones((3, 3))
  return estimated

In [14]:
for name in test_filenames:
  img = cv.imread(name)
  if img.any() == None:
        print("invalid image")
  noise_shape = (img.shape[0], img.shape[1])
  # assuming noise_sigma based on lena... should change the estimated value
  # based on the new image 
  noise = np.ones(shape=noise_shape) * noise_sigma**2

  kernel = estimate_blur_kernel(img)
  wiener_result = recover(img=img, h=kernel, sn=noise, wiener_inverse=True)
  inverse_result = recover(img=img, h=kernel, wiener_inverse=False)
  stacked = np.hstack((wiener_result, inverse_result))
  cv2_imshow('wiener vs inverse', stacked)
  cv.waitKey(0)
  cv.destroyAllWindows()
  print('Wiener vs Inverse')

Wiener vs Inverse
Wiener vs Inverse
Wiener vs Inverse


### Blind Deconvolution

Blind Deconvolution (Deconvolución a ciegas) consiste en un algoritmo que reconstruye la imagen sin necesidad de tener el modelo exacto de la función "PSF" (es por esto que se le da el nombre de "a ciegas"). El algortimo parte de una función PSF "a priori", o supone una forma de función PSF, pero luego la ajusta según la imagen de entrada. Para ajustar la función PSF, el algoritmo suele usar las secciones más brillantes de la imagen, que se vieron menos afectadas por el nivel de ruido.
Para la implementación, se utilizó la función "deconvblind" de Matlab, a la que se la paso la sencilla función PSF a priori como una matriz de 12x12 de unos. Con esto logro obtenerse resultados aceptables.

In [15]:
matlab_code_img = cv.imread('deconvCodeMATLAB.png')
#print(matlab_code_img)
cv.imshow('matlab code deconvblind', matlab_code_img)
cv.waitKey(0)
cv.destroyAllWindows()

In [38]:
in_image_blind = cv.imread('image_in_deconvblind.bmp') 
out_image_blind = cv.imread('image_out_deconvblind.bmp') 
in_out_images = np.hstack((in_image_blind, out_image_blind))
cv.imshow('in image -LEFT- vs out -RIGHT-', in_out_images)
cv.waitKey(0)
cv.destroyAllWindows()

### Motion Blur

Para simular el motion blur que degrada una imagen debido a un movimiento de una cámara al momento de tomar la foto, se realizan kernels con unos en una columna (en caso de motion vertical) o en una fila (en caso de blur horizonral).

In [32]:
kernel_h_motion = np.zeros((21, 21))
kernel_v_motion = np.zeros((21, 21))

k = 1 / 21

kernel_h_motion[2, :] = k

kernel_v_motion[:, 2] = k



#print(kernel_v_motion)
#print(kernel_h_motion)

#### Vertical Motion

In [31]:
original = cv.imread(filename=filename, flags=cv.IMREAD_COLOR)
vertical_moved = cv.filter2D(original, -1, kernel_v_motion)
noise = np.ones(shape=(256, 256)) * (noise_sigma/10)**2
#vertical_recovered = recover(vertical_moved, kernel_v_motion, sn= noise, wiener_inverse=True, show = False)
vertical_recovered = recover(vertical_moved, kernel_v_motion, sn= noise, wiener_inverse=False, show = False)
in_out_images = np.hstack((original, vertical_moved, vertical_recovered))
cv.imshow('original - LEFT- moved image -CENTER- vs recovered -RIGHT-', in_out_images)
cv.waitKey(0)
cv.destroyAllWindows()

#### Horizontal Motion

In [30]:

horizontal_moved = cv.filter2D(original, -1, kernel_h_motion)
noise = np.ones(shape=(256, 256)) * (noise_sigma/10)**2
#horizontal_recovered = recover(horizontal_moved, kernel_h_motion, sn= noise, wiener_inverse=True, show = False)
horizontal_recovered = recover(horizontal_moved, kernel_h_motion, sn= noise, wiener_inverse=False, show = False)
in_out_images = np.hstack((original, horizontal_moved, horizontal_recovered))
cv.imshow('original - LEFT- moved image -CENTER- vs recovered -RIGHT-', in_out_images)
cv.waitKey(0)
cv.destroyAllWindows()